In [4]:
import sys
sys.path.append("..")  # Adds the parent directory to the Python path if needed

In [6]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd


In [5]:
from credentials import SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, REDIRECT_URI

In [16]:
# Initialize Spotipy with authentication
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id=SPOTIFY_CLIENT_ID,
    client_secret=SPOTIFY_CLIENT_SECRET,
    redirect_uri=REDIRECT_URI,
    scope='playlist-read-private playlist-read-collaborative user-read-private user-read-email'

))


In [10]:
def get_playlist_id_from_url(url):
    """Extract playlist ID from Spotify URL"""
    if 'open.spotify.com/playlist/' in url:
        return url.split('/playlist/')[1].split('?')[0]
    return None

In [11]:
def get_playlist_tracks(playlist_id):
    """Get all tracks from a specific playlist"""
    tracks = []
    results = sp.playlist_tracks(playlist_id)
    while results:
        tracks.extend(results['items'])
        results = sp.next(results) if results['next'] else None
    return tracks


In [12]:
def get_track_details(track):
    """Extract relevant details from a track"""
    return {
        'track_name': track['track']['name'],
        'artist_name': ', '.join([artist['name'] for artist in track['track']['artists']]),
        'album_name': track['track']['album']['name'],
        'release_date': track['track']['album']['release_date'],
        'duration_ms': track['track']['duration_ms'],
        'popularity': track['track']['popularity'],
        'track_id': track['track']['id'],
        'explicit': track['track']['explicit'],
        'added_at': track.get('added_at', ''),
        'playlist_url': f"https://open.spotify.com/track/{track['track']['id']}"
    }


In [13]:
def get_audio_features(track_ids):
    """Get audio features for multiple tracks"""
    features = []
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        features.extend(sp.audio_features(batch))
    return features

In [14]:
def main():
    # Get playlist URLs from user
    playlist_urls = input("Enter Spotify playlist URLs (comma-separated): ").split(',')
    
    all_tracks = []
    for url in playlist_urls:
        url = url.strip()
        playlist_id = get_playlist_id_from_url(url)
        if not playlist_id:
            print(f"Invalid Spotify playlist URL: {url}")
            continue
        
        try:
            playlist = sp.playlist(playlist_id)
            print(f"\nProcessing playlist: {playlist['name']}")
            
            tracks = get_playlist_tracks(playlist_id)
            print(f"Found {len(tracks)} tracks")
            
            for track in tracks:
                if track['track']:  # Skip unavailable tracks
                    all_tracks.append(track)
        except Exception as e:
            print(f"Error processing {url}: {str(e)}")
            continue

    # Get track details
    track_details = [get_track_details(track) for track in all_tracks if track['track']]
    track_ids = [td['track_id'] for td in track_details]

    # Get audio features
    audio_features = get_audio_features(track_ids)
    
    # Combine data
    for td, af in zip(track_details, audio_features):
        if af:
            td.update({
                'danceability': af['danceability'],
                'energy': af['energy'],
                'key': af['key'],
                'loudness': af['loudness'],
                'mode': af['mode'],
                'speechiness': af['speechiness'],
                'acousticness': af['acousticness'],
                'instrumentalness': af['instrumentalness'],
                'liveness': af['liveness'],
                'valence': af['valence'],
                'tempo': af['tempo'],
                'time_signature': af['time_signature']
            })

    # Create DataFrame and save to CSV
    df = pd.DataFrame(track_details)
    df.to_csv('spotify_playlist_data.csv', index=False)
    print(f"\nExported {len(df)} tracks to spotify_playlist_data.csv")

In [17]:
if __name__ == '__main__':
    main()


Processing playlist: ICONIC TIKTOK EDIT AUDIOS


HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=6l8mgVN9Xf1hiDIFGA6CTE,0R8bei2Q3U688hu4XEwMq3,0lEjxUUlKqjqXrVlIHFduD,7i5dJM7fhHppWwJoAX4bZ3,3ia3dJETSOllPsv3LJkE35,5QysgWndNifZvQ6RSjZVcZ,4StJ0qBDOUtbLGLcFXJCcS,4KZAsdL0PeGvmPbSr0hyG4,5qpXZ45eZA3VX3qe76tmqh,28XDE6yXI6Bp4U3nLSGqzp,3JTjLyrnevl9ASw3ayGO2P,7KA4W4McWYRpgf0fWsJZWB,4T5Y25pWs2pVZVcFIqFuf8,17OqI90oTFZ3J8PVu6j07V,7bn8hQ1PW06VuphijGLJ40,3WMbD1OyfKuwWDWMNbPQ4g,4LrHrb9fZ39fQsMyUdrQXO,5s7m2xNZWgz5FqVSIvJcGA,5udNN5hwk870U9dk7a0nHe,4ixD6bSwnSlzuBrd2c80vI,6WqiXCXs2wb3eEL9TSrgXC,08ljt6NCNAgewQaMya1URa,35mBYyB61qTcGGj7MXIRM1,2obxSEtNEcxUss3EkamUeL,40gk32E7YaTFoQwDIWv2SY,2lnzGkdtDj5mtlcOW2yRtG,63vL5oxWrlvaJ0ayNaQnbX,3RKjTYlQrtLXCq5ncswBPp,5ka2ajep9OAvU5Sgduhiex,2pJmSaPh5lMe4npn0Z6m8A,0yNiaePZow0ycdrmLV0J7y,6cSDNYoWlMxzVfJLn6H301,3AN5LEspFZQdQUFUUqsDct,7lWUJpax919G4JdaFEVmCy,0Hb2KGmwZez23POGBw2Xc2,7lWGmPIhx5lmk0MQj4FizV,4aauSKokYE3lInHTJrf8OW,70WFvp0yF9m1asIb60gKcj,1SAkL1mYNJlaqnBQxVZrRl,5cZqsjVs6MevCnAkasbEOX,4l1R5g85v3b

Found 127 tracks


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=6l8mgVN9Xf1hiDIFGA6CTE,0R8bei2Q3U688hu4XEwMq3,0lEjxUUlKqjqXrVlIHFduD,7i5dJM7fhHppWwJoAX4bZ3,3ia3dJETSOllPsv3LJkE35,5QysgWndNifZvQ6RSjZVcZ,4StJ0qBDOUtbLGLcFXJCcS,4KZAsdL0PeGvmPbSr0hyG4,5qpXZ45eZA3VX3qe76tmqh,28XDE6yXI6Bp4U3nLSGqzp,3JTjLyrnevl9ASw3ayGO2P,7KA4W4McWYRpgf0fWsJZWB,4T5Y25pWs2pVZVcFIqFuf8,17OqI90oTFZ3J8PVu6j07V,7bn8hQ1PW06VuphijGLJ40,3WMbD1OyfKuwWDWMNbPQ4g,4LrHrb9fZ39fQsMyUdrQXO,5s7m2xNZWgz5FqVSIvJcGA,5udNN5hwk870U9dk7a0nHe,4ixD6bSwnSlzuBrd2c80vI,6WqiXCXs2wb3eEL9TSrgXC,08ljt6NCNAgewQaMya1URa,35mBYyB61qTcGGj7MXIRM1,2obxSEtNEcxUss3EkamUeL,40gk32E7YaTFoQwDIWv2SY,2lnzGkdtDj5mtlcOW2yRtG,63vL5oxWrlvaJ0ayNaQnbX,3RKjTYlQrtLXCq5ncswBPp,5ka2ajep9OAvU5Sgduhiex,2pJmSaPh5lMe4npn0Z6m8A,0yNiaePZow0ycdrmLV0J7y,6cSDNYoWlMxzVfJLn6H301,3AN5LEspFZQdQUFUUqsDct,7lWUJpax919G4JdaFEVmCy,0Hb2KGmwZez23POGBw2Xc2,7lWGmPIhx5lmk0MQj4FizV,4aauSKokYE3lInHTJrf8OW,70WFvp0yF9m1asIb60gKcj,1SAkL1mYNJlaqnBQxVZrRl,5cZqsjVs6M